In [ ]:
pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa;
import matplotlib.pyplot as plt
import random

In [ ]:
learning_rate = 0.0005
weight_decay = 0.0001
batch_size = 128
num_epochs = 60
image_size = 256  # We'll resize input images to this size
patch_size = image_size // 8  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 16
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier
num_classes = 2
input_size = 256
#input_shape = (input_size,input_size,1)

In [ ]:
def load_and_preprocess_image(image_path,image_size=(image_size,image_size)):
    image = cv2.imread(image_path)
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Resize to the desired image size
    resized_image = cv2.resize(gray_image, image_size)
    # Reshape to (64, 64, 1) for grayscale
    reshaped_image = resized_image.reshape((*image_size, 1))
    # Normalize pixel values to [0, 1]
    normalized_image = reshaped_image / 255.0
    return normalized_image

def parse_data(root_directory):
    # Initialize empty lists to store image data and labels
    x = []
    y = []

    # Traverse the directory tree
    for root, dire, _ in os.walk(root_directory):
        for d in dire:
            subdir = os.path.join(root,d)
            clas = subdir.split('/')[-1]
            #print(clas)
            if clas == 'AD':
                label = 1  # Positive class
            elif clas == 'NC':
                label = 0  # Negative class
            else:
                continue  # Skip other directories
            for _,_,filenames in os.walk(subdir):
                for fn in filenames:
                    # Get the full path of the image file            
                    image_path = os.path.join(subdir, fn)
                    
                    # Load and preprocess the image
                    #image_data = load_and_preprocess_image(image_path)

                    # Append the data and label to the lists
                    x.append(image_path)
                    y.append(label)
    d = list(zip(x,y))
    random.shuffle(d)
    x,y=zip(*d)
    x=list(x)
    y=list(y)
    return (x,y)

In [ ]:
def d_augment(x):
    original_height, original_width = tf.shape(x)[0], tf.shape(x)[1]
    # Data augmentation: Random horizontal flip
    #if tf.random.uniform(()) > 0.5:
        #x = tf.image.flip_left_right(x)

    # Data augmentation: Random rotation (90, 180, or 270 degrees)
    #rotation_angle = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32) * 90
    #x = tf.image.rot90(x, k=rotation_angle // 90)
    
    row_sum = tf.reduce_sum(x,axis=0)
    col_sum = tf.reduce_sum(x,axis=1)

    minr,maxr = tf.where(tf.math.greater(row_sum,0))[0][0],tf.where(tf.math.greater(row_sum,0))[-1][0]
    minc,maxc = tf.where(tf.math.greater(col_sum,0))[0][0],tf.where(tf.math.greater(col_sum,0))[-1][0]

    x = x[minc:maxc+1,minr:maxr+1,:]

    x = tf.image.resize(x, (image_size, image_size))
    return x
def tf_parse(x,y):
    def _parse(x,y):
        x = load_and_preprocess_image(x.decode())
        y = y
        x=d_augment(x)
        return x,y
    x,y=tf.numpy_function(_parse,[x,y],[tf.float32,tf.int32])
    x.set_shape([image_size,image_size,1])
    y.set_shape([])
    return x,y
def tf_dataset(X,Y,batch_size=batch_size):
    dataset=tf.data.Dataset.from_tensor_slices((X,Y))
    dataset=dataset.map(tf_parse)
    dataset=dataset.batch(batch_size)
    dataset=dataset.prefetch(10)
    return dataset

In [ ]:
train_dir = '/kaggle/input/adni-preprocessed/AD_NC/train/'
test_dir = '/kaggle/input/adni-preprocessed/AD_NC/test/'
(x_train,y_train) = parse_data(train_dir)
train_set = tf_dataset(x_train,y_train,batch_size=batch_size)
(x_test,y_test) = parse_data(test_dir)
test_set = tf_dataset(x_test,y_test,batch_size =len(y_test))

In [ ]:
print(f"x_train shape: {len(x_train)} - y_train shape: {len(y_train)}")
print(type(x_train[0]))
#print(y_train)
#print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
x=load_and_preprocess_image(x_train[0])

row_sum = tf.reduce_sum(x,axis=0)
col_sum = tf.reduce_sum(x,axis=1)

minr,maxr = tf.where(tf.math.greater(row_sum,0))[0][0],tf.where(tf.math.greater(row_sum,0))[-1][0]
minc,maxc = tf.where(tf.math.greater(col_sum,0))[0][0],tf.where(tf.math.greater(col_sum,0))[-1][0]

x = x[minc:maxc+1,minr:maxr+1,:]

x = tf.image.resize(x, (256, 256),preserve_aspect_ratio=False)

plt.imshow(x)
plt.show()


https://keras.io/examples/vision/image_classification_with_vision_transformer/

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
#data_augmentation.layers[0].adapt(train_set.take(1))


In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(x)
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([x]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 1))
    plt.imshow(patch_img.numpy())
    plt.axis("off")

In [ ]:
fig,axs = plt.subplots(1, 5)

for imag, label in train_set.take(1):  #n=number of batches
    for i in range(0,5):
        axs[i].imshow(imag[i,...])
plt.show()

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=(image_size,image_size,1))
    # Augment data.
    #augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.3
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.3)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            #keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/kaggle/working/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    #model.load_weights(checkpoint_filepath)

    history = model.fit(
        train_set,
        batch_size=batch_size,
        epochs=num_epochs,
        callbacks=[checkpoint_callback],
    )
    
    _, acc = model.evaluate(test_set)
    fl = f"/kaggle/working/model_{image_size}bit_acc{acc:.4f}.h5"
    vit_classifier.save_weights(fl)
    
    #model.load_weights(checkpoint_filepath)

    return history

vit_classifier = create_vit_classifier()
#vit_classifier.load_weights('/kaggle/working/model_256bit_acc0.7308.h5')
history = run_experiment(vit_classifier)

In [ ]:
loss,acc = vit_classifier.evaluate(test_set)

In [ ]:
fl = f"/kaggle/working/model_{image_size}bit_acc{acc:.4f}.h5"

vit_classifier.save_weights(fl)